In [ ]:
import h5py
import torch
from torch.utils.data import Dataset, DataLoader

class ECGDataset(Dataset):
    def __init__(self, file_path, data_key='X_train', label_key='y_train'):
        self.file_path = file_path
        self.data_key = data_key
        self.label_key = label_key
        with h5py.File(file_path, 'r') as file:
            self.length = file[data_key].shape[0]

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        with h5py.File(self.file_path, 'r') as file:
            data = file[self.data_key][idx]
            label = file[self.label_key][idx]
        return torch.tensor(data, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)


In [ ]:
import torch.nn as nn
import numpy as np
class ECGModel(nn.Module):
    def __init__(self, kernel_size, num_cnn_layers, num_lstm_units):
        super(ECGModel, self).__init__()
        self.cnn_layers = nn.Sequential()
        
        for i in range(num_cnn_layers):
            self.cnn_layers.add_module(f"conv{i+1}", nn.Conv1d(in_channels=12 if i == 0 else 32, out_channels=32, kernel_size=kernel_size))
            self.cnn_layers.add_module(f"relu{i+1}", nn.ReLU())
            self.cnn_layers.add_module(f"maxpool{i+1}", nn.MaxPool1d(kernel_size=2))

        self.lstm = nn.LSTM(input_size=32, hidden_size=num_lstm_units, batch_first=True)
        self.fc = nn.Linear(num_lstm_units, 95)  # Assuming 77 classes

    def forward(self, x):
#         print("Mean:",torch.mean(x[0]))
#         print("Std:",torch.std(x[0]))
        x = self.cnn_layers(x)
#         print(x.shape)
        x = np.swapaxes(x,1,2)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  # Taking the last LSTM output
        return torch.sigmoid(x)


In [73]:
# def train_model(model, dataloader, num_epochs, learning_rate):
#     criterion = nn.BCELoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#     for epoch in range(num_epochs):
#         for data, labels in dataloader:
#             optimizer.zero_grad()
#             outputs = model(data)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#         print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")


In [7]:
def train_and_validate(model, train_loader, val_loader, num_epochs, learning_rate):
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        # Training phase
        for data, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(data)
#             print(outputs)
#             print("Nan:",torch.isnan(outputs).any())
            #print(labels)
#             if( torch.isnan(outputs).any()):
#                 mask = np.ones(outputs.shape[0], dtype=bool)
#                 index = torch.where(torch.isnan(outputs))
#                 mask[np.unique(index[0])] = False
#                 loss = criterion(outputs[mask], labels[mask])
#                 print(loss.item())
# #                 print(outputs[mask],labels[mask])
#             else:    
            loss = criterion(outputs, labels)
#                 print(loss.item())
            loss.backward()
            optimizer.step()
        print(loss.item())
        model.eval()
        print("Trainign Loss",loss.item())
        # Validation phase
        with torch.no_grad():
            val_loss = 0
            for data, labels in val_loader:
                outputs = model(data)
#                 if()
#                 if( torch.isnan(outputs).any()):
#                     mask = np.ones(outputs.shape[0], dtype=bool)
#                     index = np.where(np.isnan(a))
#                     mask[np.unique(index[0])] = False
#                     val_loss += criterion(outputs[mask], labels[mask])
# #                     print(outputs[mask],labels[mask])
#                 else:    
                val_loss += criterion(outputs, labels)
                 

        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item()}, Validation Loss: {val_loss}")



In [ ]:
# Loaders
train_dataset = ECGDataset('output2_datasets_Norm_withoutZeros.hdf5', 'X_train', 'y_train')
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = ECGDataset('output2_datasets_Norm_withoutZeros.hdf5', 'X_val', 'y_val')
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
model = ECGModel(5,5,5)
# Train and validate the model
print("Started Training")
train_and_validate(model, train_loader, val_loader, num_epochs=10, learning_rate=0.00001)


Started Training


In [9]:
train_dataset = ECGDataset('output2_datasets.hdf5', 'data', 'class')
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [10]:
a = [[0.5139, 0.4720, 0.4192, 0.5093, 0.5618, 0.5509]]
b = [[0.5139, 0.4720, 0.4192, 0.5093, 0.5618, 0.5509]]
a = torch.tensor(a)
b = torch.tensor(b)
# loss = nn.BCELoss(a,b)
loss = nn.BCELoss()
m = nn.Sigmoid()
# input = torch.randn(1, 2, requires_grad=True)
# target = torch.rand(1, 2, requires_grad=False)
print(input)
print(a)
print(target)
loss(a, b)

<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7fbc51e42df0>>
tensor([[0.5139, 0.4720, 0.4192, 0.5093, 0.5618, 0.5509]])


NameError: name 'target' is not defined

In [65]:
model = ECGModel(5,5,5)
dataset = ECGDataset('output2_datasets_normalized.hdf5')
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
train_model(model,dataset,20,1e-4)

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [70]:
a =torch.tensor([[1,2,3,4],[np.nan,np.nan,np.nan,np.nan],[np.nan,np.nan,np.nan,np.nan]],dtype=torch.float)

b = torch.tensor([[1,0,0,0],[0,1,0,0],[0,1,0,0]],dtype=torch.float)
loss1 = nn.BCELoss()
m = nn.Sigmoid()
mask = np.ones(a.shape[0], dtype=bool)
index = np.where(np.isnan(a))
mask[np.unique(index[0])] = False

print(np.unique(index[0]))
loss1(m(a[mask,:]),b[mask,:])

[1 2]


tensor(2.3767)

In [63]:
mask

array([ True, False, False])

In [68]:
a[mask,:]

tensor([[1., 2., 3., 4.]])

In [69]:
b[mask,:]

tensor([[1., 0., 0., 0.]])